# Embeddings

Привет! В этом домашнем задании мы с помощью эмбеддингов решим задачу семантической классификации твитов.

Для этого мы воспользуемся предобученными эмбеддингами word2vec.

Для начала скачаем датасет для семантической классификации твитов:

Импортируем нужные библиотеки:

In [1]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

import re
from nltk.corpus import stopwords
from torch import nn

In [2]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные:

In [4]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [61]:
examples = data["text"].sample(10)
print("\n".join(examples))

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


Как видим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [5]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [6]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

@ chrishasboobs ahhh i hope your ok !!!


In [7]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

chrishasboobs ahhh hope your


Загрузим предобученную модель эмбеддингов. 

Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data.

Данная модель выдает эмбеддинги для **слов**. Строить по эмбеддингам слов эмбеддинги предложений мы будем ниже.

In [8]:
word2vec = api.load("word2vec-google-news-300")

In [9]:
type(word2vec)

gensim.models.keyedvectors.KeyedVectors

In [67]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(300,)


Нормализуем эмбеддинги, прежде чем обучать на них сеть. 
(наверное, вы помните, что нейронные сети гораздо лучше обучаются на нормализованных данных)

In [68]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

(300,)
[False, False, False, False, False, False, False, False, False, False, False, False]


In [69]:
np.array(norm_emb_line).shape

(12, 300)

Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [10]:
stopWords = set(stopwords.words('english'))

In [11]:
from torch.utils.data import Dataset, random_split


class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        text_ = re.sub(f'[{string.punctuation}]', ' ', text)
        text_ = re.findall('\w+', text_)
        #text_ = " ".join(filter(lambda x:x[0]!='@', text_))    
        return [word.lower() for word in text_ if word.lower() not in stopWords]

    def get_embeddings_(self, tokens):
        embeddings = np.array([((word2vec[x] - self.mean) / self.std) \
                               for x in tokens if x in word2vec]) # Получи эмбеддинги слов и усредни их
        if len(embeddings) != 0:
            embeddings = sum(embeddings) / len(embeddings)
        
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            #embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(1, -1)
        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [12]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

Отлично, мы готовы с помощью эмбеддингов слов превращать твиты в векторы и обучать нейронную сеть.

Превращать твиты в векторы, используя эмбеддинги слов, можно несколькими способами. А именно такими:

## Average embedding (2 балла)
---
Это самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предлоежнии (которые остались после токенизации и удаления коротких слов, конечно). 

In [73]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


In [74]:
#np.swapaxes(examples["features"],0, 1)

In [75]:
#np.array(examples["targets"]).shape

Давайте сделаем визуализацию полученных векторов твитов тренировочного (dev) датасета. Так мы увидим, насколько хорошо твиты с разными target значениями отделяются друг от друга, т.е. насколько хорошо усреднение эмбеддингов слов предложения передает информацию о предложении.

Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Если хотите, можете вместо PCA использовать TSNE: так у вас получится более точная проекция на плоскость (а значит, более информативная, т.е. отражающая реальное положение векторов твитов в пространстве). Но TSNE будет работать намного дольше.

In [76]:
from sklearn.decomposition import PCA


pca = PCA(n_components=2)
examples["transformed_features"] = pca.fit_transform(examples['features'], examples['targets'])# Обучи PCA на эмбеддингах слов

In [77]:
# from sklearn.decomposition import PCA


# pca = PCA(n_components=2)
# pca.fit(examples["features"], examples["targets"])
#pca.fit(np.swapaxes(examples["features"],0, 1),np.array(examples["targets"]).reshape(-1, 1))#['features'], examples['targets'])

In [78]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [79]:
# draw_vectors(
#     examples["transformed_features"][:, 0], 
#     examples["transformed_features"][:, 1], 
#     color=[["red", "blue"][t] for t in examples["targets"]]
#     )

Скорее всего, на визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [80]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 0

def average_emb(batch):
    features = np.array([np.mean(b["feature"], axis=0) for b in batch])
    targets = np.array([b["target"] for b in batch]).astype(float)

    #return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}
    return (torch.FloatTensor(features), torch.LongTensor(targets))


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

Определим функции для тренировки и теста модели:

In [28]:
from sklearn.metrics import accuracy_score

In [29]:
from tqdm.notebook import tqdm


def training(model, optimizer, criterion, scheduler, train_loader, epoch, device="cuda"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        features = batch[0].to(device)
        targets = batch[1].to(torch.float32).to(device)
        
        optimizer.zero_grad()
        predictions = model(features) # Получи предсказания модели
        #print(type(targets[0][0]))
        loss = criterion(predictions, targets) # Посчитай лосс
        loss.backward() # Обнови параметры модели
        optimizer.step()
        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss.cpu():.4}")
    scheduler.step()
    

def testing(model, criterion, test_loader, device="cuda"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch[0].to(device)
            targets = batch[1].to(torch.float32).to(device)

            predictions = model(features) # Получи предсказания модели
            #print(type(targets[0]))
            loss = criterion(predictions, targets) # Посчитай лосс
            targets = torch.round(targets.cpu())
            predictions = torch.round(predictions.cpu())
            acc = accuracy_score(targets, predictions) # Посчитай точность модели

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию. Вы можете сами выбрать количество слоев в нейронной сети, ваш любимый оптимизатор и целевую функцию.


In [30]:
layer_size = 1024

In [31]:
class NN(nn.Module):
  
    def __init__(self, n_classes, dim):
        super().__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(dim,  layer_size),
            nn.BatchNorm1d(layer_size),
            nn.ReLU()
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(layer_size,  layer_size),
            nn.BatchNorm1d(layer_size),
            nn.ReLU()
        )
        
        self.fc3 = nn.Sequential(
            nn.Linear(layer_size,  layer_size),
            nn.BatchNorm1d(layer_size),
            nn.ReLU()
        )
        
        self.out = nn.Linear(layer_size,  n_classes)
        self.sigm = nn.Sigmoid()
    def forward(self, x):
        # print(x)
        # x = x.permute(0, 2, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x_out = self.out(x)
        return torch.reshape(self.sigm(x_out), (-1,))
        #return torch.reshape(x_out, (-1,))

In [87]:
!nvidia-smi

In [88]:
import torch.nn as nn
from torch.optim import Adam


# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
num_classes = 1
lr = 1e-2
num_epochs = 3

model = NN(num_classes, 300) # Твоя модель
model = model.cuda()
criterion =  nn.BCELoss() # Твой лосс
optimizer = Adam(model.parameters(), lr = lr) # Твой оптимайзер

Наконец, обучим модель и протестируем её.

После каждой эпохи будем проверять качество модели на валидационной части датасета. Если метрика стала лучше, будем сохранять модель. **Подумайте, какая метрика (точность или лосс) будет лучше работать в этой задаче?** 

In [89]:
best_metric = np.inf
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma=0.1)
for e in range(num_epochs):
    training(model, optimizer, criterion, scheduler, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.4918893357515335, 'Test Acc': 0.75840625}


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.47732950627803805, 'Test Acc': 0.767609375}


Epoch 5. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.47723623836040496, 'Test Acc': 0.76756640625}


In [90]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.47905410421542083, 'Test Acc': 0.7660025209664537}


## Embeddings for unknown words (8 баллов)

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

- Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. (4 балла)
- Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf. (4 балла)

Реализуйте оба варианта **ниже**. Напишите, какой способ сработал лучше и ваши мысли, почему так получилось.

In [91]:
class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        text_ = re.sub(f'[{string.punctuation}]', ' ', text)
        text_ = re.findall('\w+', text_) 
        return [word.lower() for word in text_ if word.lower() not in stopWords]

    def get_embeddings_(self, tokens):
        embeddings = []
        ind = [] 
        for i, word in enumerate(tokens):
            if word in word2vec:
                embeddings.append(word2vec[word])
                ind.append(i)
        for i, word in enumerate(tokens):
            if word not in word2vec:
                vec = np.zeros(300)
                lower_bound = max([-1] + [x for x in ind if x < i])
                upper_bound = min([len(tokens)] + [x for x in ind if x > i])
                nb = False 
                if (lower_bound > -1):
                    vec += word2vec[tokens[lower_bound]]
                    nb = True 
                if (upper_bound < len(tokens)):
                    vec += word2vec[tokens[upper_bound]]
                    nb = True
                if nb:
                    embeddings.append(vec)
        #embeddings = np.array([word2vec[x] for x in tokens if x in word2vec]) # Получи эмбеддинги слов и усредни их
        if len(embeddings) != 0:
            embeddings = sum(embeddings) / len(embeddings)
        
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            #embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(1, -1)
        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [92]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

In [93]:
num_workers = 0

def average_emb(batch):
    features = np.array([np.mean(b["feature"], axis=0) for b in batch])
    targets = np.array([b["target"] for b in batch]).astype(float)

    #return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}
    return (torch.FloatTensor(features), torch.LongTensor(targets))


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [94]:
# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
num_classes = 1
lr = 1e-2
num_epochs = 3

model = NN(num_classes, 300) # Твоя модель
model = model.cuda()
criterion =  nn.BCELoss() # Твой лосс
optimizer = Adam(model.parameters(), lr = lr) # Твой оптимайзер

In [95]:
best_metric = np.inf
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma=0.5)
for e in range(num_epochs):
    training(model, optimizer, criterion, scheduler, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5054535182714462, 'Test Acc': 0.74821484375}


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.48566075456142427, 'Test Acc': 0.76310546875}


Epoch 5. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.484594456076622, 'Test Acc': 0.76433203125}


In [96]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.48383807488523733, 'Test Acc': 0.7644612370207667}


## tfidf

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(np.array(data["text"]))
len(vectorizer.get_feature_names_out())

684358

In [15]:
from sklearn.decomposition import TruncatedSVD

In [16]:
X = X.transpose()

In [17]:
svd = TruncatedSVD(n_components=300, n_iter=25)
tr_emb = svd.fit_transform(X)

In [88]:
from torch.utils.data import Dataset, random_split


class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str,\
                 word2vec: gensim.models.Word2Vec, tr_emb: np.ndarray, voc):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)
        
        self.tr_mean = np.mean(tr_emb, axis=0)
        self.tr_std = np.std(tr_emb, axis=0)
        
        self.voc = voc
        self.tr_emb = tr_emb

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        text_ = re.sub(f'[{string.punctuation}]', ' ', text)
        text_ = re.findall('\w+', text_)

        return [word.lower() for word in text_ if word.lower() not in stopWords]

    def get_embeddings_(self, tokens):
        # Получи эмбеддинги слов и усредни их
#         embeddings = np.array([((word2vec[x] - self.mean) / self.std) \
#                                for x in tokens if x in word2vec]) 
        embeddings = []
        #print(tokens)
        for w in tokens:
            if w not in self.voc :
                continue
            nw = (self.tr_emb[self.voc[w]] - self.tr_mean) / self.tr_std
            if w in self.word2vec:
                nw += (self.word2vec[w] - self.mean) / self.std 
            embeddings.append(nw)
            
        if len(embeddings) != 0:
            embeddings = sum(embeddings) / len(embeddings)
        
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            #embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(1, -1)
        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [95]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec, tr_emb = tr_emb, voc = vectorizer.vocabulary_)

In [97]:
from torch.utils.data import DataLoader
batch_size = 1024
num_workers = 0

def average_emb(batch):
    features = np.array([np.mean(b["feature"], axis=0) for b in batch])
    targets = np.array([b["target"] for b in batch]).astype(float)

    #return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}
    return (torch.FloatTensor(features), torch.LongTensor(targets))


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [98]:
import torch.nn as nn
from torch.optim import Adam
vector_size = dev.word2vec.vector_size
num_classes = 1
lr = 1e-2
num_epochs = 3

model = NN(num_classes, 300) # Твоя модель
model = model.cuda()
criterion =  nn.BCELoss() # Твой лосс
optimizer = Adam(model.parameters(), lr = lr) # Твой оптимайзер

In [ ]:
best_metric = np.inf
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma=0.3)
for e in range(num_epochs):
    training(model, optimizer, criterion, scheduler, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5246816556453705, 'Test Acc': 0.73408984375}


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

...

Epoch 15. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5095833743810654, 'Test Acc': 0.7504140625}


Epoch 16. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]